# Data Gathering

This notebook gathers the meta-data for the evaluation. It runs the various respiratory extraction methods.

In [ ]:
import os

import numpy as np

import respiration.dataset as repository

dataset = repository.from_default()
subjects = dataset.get_subjects()
subjects

In [ ]:
# The scenarios in which the subjects were recorded
# scenarios = dataset.get_scenarios()
# scenarios = [
#     '101_natural_lighting',
#     # '102_artificial_lighting',
#     '103_abrupt_changing_lighting',
#     '104_dim_lighting_auto_exposure',
#     # '106_green_lighting',
#     # '107_infrared_lighting',
#     '201_shouldercheck',
#     '202_scale_movement',
#     '203_translation_movement',
#     # '204_writing'
# ]
scenarios = [
    '101_natural_lighting',
]
scenarios

In [ ]:
results_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals-2024-04-20')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

## Extract the ground truth signals

In [ ]:
import pandas as pd
import respiration.utils as utils

ground_truth_signals = []

for subject in subjects:
    for idx, scenario in enumerate(scenarios):
        gt_signal, gt_sampling_rate = dataset.get_ground_truth_rr_signal(subject, scenario)
        ground_truth_signals.append({
            'subject': subject,
            'scenario': scenario,
            'signal': gt_signal.tolist(),
            'sampling_rate': gt_sampling_rate,
        })

# Save the ground truth signals as a JSON
json_path = os.path.join(results_dir, 'ground_truth.json')
utils.write_json(json_path, ground_truth_signals)

# Save the ground truth signals as a CSV
df_gt = pd.DataFrame(ground_truth_signals)
csv_path = os.path.join(results_dir, 'ground_truth.csv')
df_gt.to_csv(csv_path, index=False)

## Conduct experiments

In [ ]:
from datetime import datetime

parameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
    'use_cgof': True,
}

evaluation_metadata = {
    'start_time': datetime.now(),
    'subjects': subjects,
    'scenarios': scenarios,
    'parameters': parameters,
}

In [ ]:
import respiration.roi as roi

yolo = roi.YOLO()


def get_rois(frame: np.ndarray) -> list[tuple[np.ndarray, str]]:
    """
    Get the regions of interest (ROIs) for the given frame
    :param frame: The frame to get the ROIs from
    :return: A list of tuples containing the ROI and the name of the ROI
    """

    regions = [
        # ROI for the full frame
        ((0, 0, frame.shape[1], frame.shape[0]), 'full')
    ]

    # Calculate the region of interest (ROI) based on the face
    faces = roi.detect_faces(frame)
    if len(faces) == 1:
        chest_roi = roi.roi_from_face(faces[0])
        regions.append((chest_roi, 'chest'))

    # Use the detected person to create a mask
    persons = yolo.detect_classes(frame, clazz='person')
    if len(persons) == 1:
        regions.append((persons[0], 'person'))

    return regions

In [ ]:
from respiration.extractor import pixel_intensity, optical_flow
from respiration.extractor.mtts_can import load_model, calculate_cutoff, preprocess_video_frames

extracted_signals = []

frame_depth = 10
mtts_model = load_model(frame_depth=frame_depth)

for subject in subjects:
    for idx, scenario in enumerate(scenarios):
        print(f'Processing {subject} - {scenario}')

        frames, params = dataset.get_video_bgr(subject, scenario, False)

        frames_gray = utils.convert_to_gray(frames)

        rois = get_rois(frames_gray[0])

        for region in rois:
            roi_area, roi_name = region

            #
            # Calculate the average pixel intensity
            #

            pi_start = datetime.now()
            pi_signal = pixel_intensity.average_pixel_intensity(frames_gray, roi=roi_area)
            extracted_signals.append({
                'subject': subject,
                'scenario': scenario,
                'method': 'pixel_intensity',
                'roi': roi_name,
                'roi_area': list(roi_area),
                'execution_time': datetime.now() - pi_start,
                'sampling_rate': params.fps,
                'signal': pi_signal.tolist(),
            })

            #
            # Calculate the optical flow
            #

            of_cgof_start = datetime.now()
            of_signal_raw = optical_flow.extract_signal(
                frames_gray,
                roi=roi_area,
                quality_level=parameters['quality_level'],
                quality_level_rv=parameters['quality_level_rv'],
                use_cgof=parameters['use_cgof'],
            )

            extracted_signals.append({
                'subject': subject,
                'scenario': scenario,
                'method': 'optical_flow',
                'roi': roi_name,
                'roi_area': list(roi_area),
                'execution_time': datetime.now() - of_cgof_start,
                'sampling_rate': params.fps,
                'signal': of_signal_raw.tolist(),
            })

        #
        # Calculate the MTTS-CAN signal
        #

        mtts_start = datetime.now()
        resized, normalized = preprocess_video_frames(frames)
        cutoff = calculate_cutoff(resized.shape[0], frame_depth)

        resized = resized[:cutoff]
        normalized = normalized[:cutoff]

        mtts_raw = mtts_model.predict(
            (resized, normalized),
            batch_size=100
        )
        extracted_signals.append({
            'subject': subject,
            'scenario': scenario,
            'method': 'mtts_can',
            'roi': 'full',
            'roi_area': [0, 0, frames.shape[2], frames.shape[1]],
            'execution_time': datetime.now() - mtts_start,
            'sampling_rate': params.fps,
            'signal': np.cumsum(mtts_raw[1]).tolist(),
        })

        # Garbage collect the frames
        del frames

In [ ]:
evaluation_metadata['end_time'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
# Save the extracted_signals as a JSON
json_path = os.path.join(results_dir, 'predictions.json')
utils.write_json(json_path, extracted_signals)

# Save the evaluation dataframe
csv_path = os.path.join(results_dir, 'predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(results_dir, 'parameters.json')
utils.write_json(json_path, evaluation_metadata)